In [1]:
import pandas as pd
import requests
import json
import threading
import os
import time
from requests.utils import quote
pd.options.mode.chained_assignment = None

## TempDF is the repository of all Query Results
Every query collects the DOI, PII (for science Direct), the title and the Journal Name

In [47]:
global TempDF

TempDF=pd.DataFrame(columns=['Query','PII','DOI','Title', 'Journal'])
TempDF=TempDF.set_index('DOI')


In [48]:
TempDF

,Query,PII,Title,Journal
DOI,,,,


## API List
Keep checking APIs before every run to see if any of them have expired

In [174]:
API=['580f859e483c28be2f96a25bab0fa0be','086e2acab5bf938029bba3818db45aff','7c86ebf119065c91ff6f6dc13309b018']

## Entery Query or Query list

In [175]:
query=['glass','adad','ads']

## Science Direct Query Search

In [182]:
header = {'User-Agent': 'Mozilla/5.0',
           'accept-encoding': 'UTF-8'} 

def science_direct_search(query, API,TempDF,start=0,count=5000):

        for star in range(start,count,25):
            url='https://api.elsevier.com/content/search/sciencedirect?query='+query+'&apiKey='+API+'&start='+str(star)+'&count=25&httpAccept=application/json'
            r=requests.get(url, headers=header,timeout=30)
            try:
                if r.status_code==200:
                    body=r.json()['search-results']['entry']
                    for item in body:
                        TempDF.loc[item['prism:doi']]=(query,item['pii'],item ['dc:title'], item['prism:publicationName'])
                    
            except Exception as e:
                print(e)
        
        return None

## Run Science Direct Search

In [179]:
science_direct_search('glass',API[0],TempDF,count=100)

In [196]:
len(TempDF)

99

## Cross Reference Query

This runs until the query is exhausted. So could take a long time per query. 
It is better to run this on the HPC/Download Server

In [37]:
def cross_reference_search(query,TempDF):
    
    cursor = "*"
    keep_paging = True
    max_rows = 1000
    
    
    base_url = 'https://api.crossref.org/works?query='

    headers = {
           'Accept': 'application/json'
         }
    
    
    while (keep_paging):
        
        try:
    
            r = requests.get(base_url + query + "&rows=" + str(max_rows) + "&cursor=" + cursor,
                                      headers=headers, timeout=30)
        
            cursor = quote(r.json()['message']['next-cursor'], safe='')

            if len(r.json()['message']['items']) == 0:
                            keep_paging = False
            
            for item in r.json()['message']['items']:
                try:
                    Journal=item['container-title'][0]
                except:
                    Journal='None'

                TempDF.loc[item['DOI']]=(query,'None',item['title'][0],Journal)
               
               
        except:
            pass
        
    return None                         

## Run Cross Reference Search

In [ ]:
cross_reference_search('magnesium alloy',TempDF)

In [ ]:
len(TempDF)

In [44]:
TempDF.head()

,Query,PII,Title,Journal
DOI,,,,
10.1016/s0099-2399(06)81890-x,cement,None,Solving endodontic isolation problems with int...,Journal of Endodontics
10.2118/178853-ms,cement,None,Improved Cement Plug Placement Method Resultin...,IADC/SPE Drilling Conference and Exhibition
10.2118/178772-ms,cement,None,New Approach In Lifting Cement In Highly Deple...,IADC/SPE Drilling Conference and Exhibition
10.2118/178793-ms,cement,None,Development of Geopolymer-based Cement Slurrie...,IADC/SPE Drilling Conference and Exhibition
10.1063/1.4973029,cement,None,Peculiarities of volume-phase characteristics ...,None


In [46]:
len(TempDF.drop_duplicates(subset='Title'))

53457

In [31]:
cursor = "*"
keep_paging = True
max_rows = 1000
query='cement'

headers = {
           'Accept': 'application/json'
         }

base_url = 'https://api.crossref.org/works?query='

In [9]:
r = requests.get(base_url + query + "&rows=" + str(max_rows) + "&cursor=" + cursor,headers=headers, timeout=30)

In [30]:
len(r.json()['message']['items'])

1000

In [1]:
while (keep_paging):
    
            r = requests.get(base_url + query + "&rows=" + str(max_rows) + "&cursor=" + cursor,
                                      headers=headers, timeout=30)
        
            cursor = quote(r.json()['message']['next-cursor'], safe='')

            if len(r.json()['message']['items']) == 0:
                            keep_paging = False
            
            for item in r.json()['message']['items']:
                
                try:
                    Journal=item['container-title'][0]
                except:
                    Journal='None'

                TempDF.loc[item['DOI']]=(query,'None',item['title'][0],Journal)

In [28]:
cursor4 = quote(r.json()['message']['next-cursor'], safe='')

In [29]:
cursor, cursor2,cursor3, cursor4

('AoJz153gnfACPxJodHRwOi8vZHguZG9pLm9yZy8xMC4xMDE2L2ouY2VtY29ucmVzLjIwMTcuMDUuMDA0',
 'AoJ448%2FP3fACPw9odHRwOi8vZHguZG9pLm9yZy8xMC4xMDE2LzAwMDgtODg0Nig5NCk5MDE2Ni14',
 'AoJ448%2FP3fACPw9odHRwOi8vZHguZG9pLm9yZy8xMC4xMDE2LzAwMDgtODg0Nig5NCk5MDE2Ni14',
 'AoJ6y%2BrkhPECPwdodHRwOi8vZHguZG9pLm9yZy8xMC4xMTE1OS9pY2NwZTE4LjExMQ%3D%3D')

In [27]:
r = requests.get(base_url + query + "&rows=" + str(max_rows) + "&cursor=" + cursor3,headers=headers, timeout=30)

In [33]:
len(TempDF)

3000

In [198]:
TempDF.to_csv('aassd')

1975

## Science Direct Example

In [184]:
#url='https://api.elsevier.com/content/search/sciencedirect?query='+query+'&apiKey='+API[0]+'&start=0&count=25&httpAccept=application/json'
#r=requests.get(url, headers=header,timeout=30)
#if r.status_code==200:
#    body=r.json() 

## Crossref Example

In [195]:
#cursor = "*"
#base_url = 'https://api.crossref.org/works?query='
#query='glass'
#max_rows=100
#headers = {
#           'Accept': 'application/json'
#         }
#    
#r = requests.get(base_url + query + "&rows=" + str(max_rows) + "&cursor=" + cursor,headers=headers, timeout=30)